In [ ]:
import yfinance as yf
import pandas as pd
import os
import time
import requests

# Define the NIFTY 50 index ticker
nifty50_ticker = '^NSEI'
csv_file = 'C:\\Users\\rusha\\OneDrive\\Desktop\\STOCK_MARKET\\merged1.csv'
required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Shares Traded', 'Turnover (₹ Cr)']

# Function to check internet connectivity
def check_internet_connection(url='http://www.google.com/', timeout=5):
    try:
        _ = requests.get(url, timeout=timeout)
        return True
    except requests.ConnectionError:
        return False

# Function to fetch historical data
def fetch_historical_data():
    try:
        nifty50_data = yf.Ticker(nifty50_ticker)
        hist = nifty50_data.history(period="5d")
        hist.reset_index(inplace=True)

        # Drop unnecessary columns
        hist = hist.drop(columns=[col for col in ['Dividends', 'Stock Splits', 'Volume'] if col in hist.columns])

        hist['Date'] = hist['Date'].dt.strftime('%d-%b-%Y')  # Format date
        return hist
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

# Function to standardize date formats in existing data
def standardize_date_format(date_str):
    for fmt in ('%d-%b-%Y', '%d-%b-%y', '%d-%m-%Y', '%Y-%m-%d'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            pass
    raise ValueError(f"No valid date format found for {date_str}")

# Function to append data to the CSV file
def append_data_to_csv(data, file):
    try:
        # Add placeholders for 'Shares Traded' and 'Turnover (₹ Cr)'
        data['Shares Traded'] = None
        data['Turnover (₹ Cr)'] = None
        
        # Ensure all required columns are present in the data
        for column in required_columns:
            if column not in data.columns:
                data[column] = None

        # Reorder columns to match the required columns
        data = data[required_columns]

        # Check if the CSV file exists
        if os.path.exists(file):
            existing_df = pd.read_csv(file)
            # Strip whitespace from column names
            existing_df.columns = existing_df.columns.str.strip()
            data.columns = data.columns.str.strip()

            # Standardize date formats for comparison
            existing_df['Date'] = existing_df['Date'].apply(standardize_date_format)
            data['Date'] = data['Date'].apply(standardize_date_format)

            # Combine existing data with new data, avoiding duplicates
            updated_df = pd.concat([existing_df, data], ignore_index=True)
            updated_df.drop_duplicates(subset='Date', keep='last', inplace=True)
        else:
            updated_df = data

        # Save the updated DataFrame to the CSV file
        updated_df.to_csv(file, index=False)
        print(f"Data appended to {file}.")
    except Exception as e:
        print(f"Error appending data to CSV: {e}")

# Run the data fetching and appending process
def run_data_update():
    if check_internet_connection():
        historical_data = fetch_historical_data()
        
        if not historical_data.empty:
            print(f"Fetched historical data:")
            print(historical_data)
            # Append the fetched historical data to CSV
            append_data_to_csv(historical_data, csv_file)
        else:
            print("Failed to fetch historical data.")
    else:
        print("No internet connection. Will retry in 10 minutes.")
        time.sleep(600)  # Wait for 10 minutes before retrying

# Infinite loop to run the update every hour
while True:
    run_data_update()
    time.sleep(3600)  # Sleep for 1 hour before running the update again


Fetched historical data:
          Date          Open          High           Low         Close
0  23-Jul-2024  24568.900391  24582.550781  24074.199219  24479.050781
1  24-Jul-2024  24444.949219  24504.250000  24307.250000  24413.500000
2  25-Jul-2024  24230.949219  24426.150391  24210.800781  24406.099609
3  26-Jul-2024  24423.349609  24861.150391  24410.900391  24834.849609
4  29-Jul-2024  24943.300781  24999.750000  24774.599609  24836.099609
Data appended to C:\Users\rusha\OneDrive\Desktop\STOCK_MARKET\merged1.csv.


C:\Users\rusha\AppData\Local\Temp\ipykernel_16160\1792334546.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([existing_df, data], ignore_index=True)


No internet connection. Will retry in 10 minutes.
Fetched historical data:
          Date          Open          High           Low         Close
0  23-Jul-2024  24568.900391  24582.550781  24074.199219  24479.050781
1  24-Jul-2024  24444.949219  24504.250000  24307.250000  24413.500000
2  25-Jul-2024  24230.949219  24426.150391  24210.800781  24406.099609
3  26-Jul-2024  24423.349609  24861.150391  24410.900391  24834.849609
4  29-Jul-2024  24943.300781  24999.750000  24774.599609  24836.099609


C:\Users\rusha\AppData\Local\Temp\ipykernel_16160\1792334546.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([existing_df, data], ignore_index=True)


Data appended to C:\Users\rusha\OneDrive\Desktop\STOCK_MARKET\merged1.csv.
Fetched historical data:
          Date          Open          High           Low         Close
0  23-Jul-2024  24568.900391  24582.550781  24074.199219  24479.050781
1  24-Jul-2024  24444.949219  24504.250000  24307.250000  24413.500000
2  25-Jul-2024  24230.949219  24426.150391  24210.800781  24406.099609
3  26-Jul-2024  24423.349609  24861.150391  24410.900391  24834.849609
4  29-Jul-2024  24943.300781  24999.750000  24774.599609  24836.099609


C:\Users\rusha\AppData\Local\Temp\ipykernel_16160\1792334546.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([existing_df, data], ignore_index=True)


Data appended to C:\Users\rusha\OneDrive\Desktop\STOCK_MARKET\merged1.csv.
Fetched historical data:
          Date          Open          High           Low         Close
0  23-Jul-2024  24568.900391  24582.550781  24074.199219  24479.050781
1  24-Jul-2024  24444.949219  24504.250000  24307.250000  24413.500000
2  25-Jul-2024  24230.949219  24426.150391  24210.800781  24406.099609
3  26-Jul-2024  24423.349609  24861.150391  24410.900391  24834.849609
4  29-Jul-2024  24943.300781  24999.750000  24774.599609  24836.099609


C:\Users\rusha\AppData\Local\Temp\ipykernel_16160\1792334546.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([existing_df, data], ignore_index=True)


Data appended to C:\Users\rusha\OneDrive\Desktop\STOCK_MARKET\merged1.csv.
